# Imports

In [2]:
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
from scipy import optimize
import sys
sys.path.append('../SEIR_full/')
sys.path.append('..')
import SEIR_full as mdl
import SEIR_full.model as mdl
import SEIR_full.calibration as mdl
import datetime as dt
from scipy.stats import poisson
from scipy.stats import binom
import copy

In [3]:
plt.style.use('dean_style.conf')

In [4]:
with (open('../Data/parameters/indices.pickle', 'rb')) as openfile:
    ind = pickle.load(openfile)

# Big Loop for total calibration:

In [5]:
mdl.population_size

array([4.88293605e-04, 9.70704678e-04, 1.68627048e-03, 2.00095182e-03,
       1.59025388e-03, 1.29208341e-03, 2.23179296e-03, 2.30168537e-03,
       2.75309706e-03, 9.27757850e-03, 8.18688662e-03, 1.42219416e-02,
       1.14282550e-02, 9.08259095e-03, 7.37961735e-03, 4.53121600e-03,
       3.16550079e-03, 2.62404564e-03, 2.13001518e-04, 3.71814082e-04,
       6.55246327e-04, 7.30334768e-04, 4.49067351e-04, 2.93094424e-04,
       6.10033502e-04, 6.88240772e-04, 8.83342569e-04, 4.04702884e-03,
       3.13586594e-03, 5.52632280e-03, 4.17124085e-03, 2.56480749e-03,
       1.67398225e-03, 1.23855287e-03, 9.46535409e-04, 8.41935886e-04,
       1.14670625e-04, 2.15525717e-04, 3.10108508e-04, 2.79301946e-04,
       2.26815331e-04, 1.61732610e-04, 2.22232979e-04, 1.76840984e-04,
       1.66379197e-04, 2.17874188e-03, 1.81773576e-03, 2.61544345e-03,
       1.59520776e-03, 1.29543521e-03, 9.23721151e-04, 4.51200291e-04,
       2.43208860e-04, 1.58580172e-04, 7.66860522e-05, 1.52178713e-04,
      

In [6]:
with open(
        '../Data/division_choice/250_to_30/250_to_30_region_mapper.pickle',
        'rb') as pickle_in:
    region_250_to_30 = pickle.load(pickle_in)

In [7]:
for key, idxs in region_250_to_30.items():
    print(key, ': ', mdl.population_size[idxs].sum()*mdl.pop_israel)

IndexError: index 4338 is out of bounds for axis 0 with size 540

In [ ]:
for scen in [mdl.num2scen(i) for i in [1]]:
    for phase in [29, 60, 90]:
        # run calibration for model parameters
        mdl.make_calibration(
            scen,
            phase,
            ind,
        )
        
        # run calibration for model tracking states
        mdl.make_track_calibration(
            scen,
            phase,
            ind,
        )

# Running manually:

In [8]:
# loads mapping 250 to 20:
with open('../Data/division_choice/250_to_30/250_to_30_region_mapper.pickle', 'rb') as pickle_in:
    region_250_to_30 = pickle.load(pickle_in)
with open('../Data/division_choice/250_to_30/250_to_30_region_age_mapper.pickle', 'rb') as pickle_in:
    region_age_250_to_30 = pickle.load(pickle_in)
with open('../Data/division_choice/250_to_30/250_to_30_region_ga_mapper.pickle', 'rb') as pickle_in:
    region_ga_250_to_30 = pickle.load(pickle_in)

# Amit Model play for initial values of calibration

## Data Loading

In [9]:
first = pd.Timestamp('2020-03-03')
last = pd.Timestamp('2020-05-01')
tot = pd.read_csv('../Data/sick/all_tests_by_age_county.csv')
pos = pd.read_csv('../Data/sick/smooth_sick_by_age_calibration_county.csv')
pos.set_index('Unnamed: 1', inplace=True)
pos.columns = ['county'] + list(pos.columns)[1:]
pos.sort_values(by='county', inplace=True)
pos = pos.pivot(columns='county')
pos.columns = pos.columns.swaplevel(0, 1)
pos = pos.reindex(pd.MultiIndex.from_product([region_250_to_30.keys(), ind.A.values()]), axis=1)
pos.index = pd.to_datetime(pos.index)
pos = pos.reindex(pd.date_range(pos.index.values.min(), pos.index.values.max(),freq='d'), fill_value=0)
pos.fillna(0, inplace=True)

begining = pos.index.values.min()
ending = pos.index.values.max()
pos = pos.reindex(pd.date_range(begining, ending,freq='d'), fill_value=0)

# Shifting data by 5 days for fit
pos_0303 = pos.loc['2020-03-03':].copy() #starting the fit from March 3rd 2020

shift_date_list = pd.date_range('2020-02-27',freq='d',periods=pos_0303.shape[0])

pos_0303['shift_date'] = shift_date_list

pos_0303.set_index('shift_date',inplace=True)
data = np.ndarray((2,len(pos_0303), len(region_age_250_to_30.keys())))
data[0] = np.zeros(pos_0303.shape) # insert the tested positive specimens only for relevant calibration period 
data[1] = pos_0303

pos_0303.iloc[-1]

11         0-4      0.026767
           5-9      0.039513
           10-19    0.131286
           20-29    0.219235
           30-39    0.151680
                      ...   
71_haredi  30-39    0.196685
           40-49    0.261476
           50-59    0.129581
           60-69    0.039337
           70+      0.018512
Name: 2020-05-01 00:00:00, Length: 270, dtype: float64

## Define and run model

In [10]:
# fit params
start = pos_0303.index[0]
end = pos_0303.index[-1]
date_lst = pd.date_range('2020-02-20',end)
len(date_lst)

72

In [11]:
scen = 'Scenario1'
model = mdl.Model_behave(
    ind=ind,
    beta_j=np.array([0.10167117, 0.10167117, 0.10167117, 0.05606748, 0.05606748,
       0.04709613, 0.04709613, 0.25676354, 0.25676354])*0.4,
    theta=2.0684200685446243,
    beta_behave=0.38,
    scen=scen,
    seasonality=True
    
)
res_mdl = model.predict(
    C=mdl.C_calibration,
    days_in_season=len(date_lst),
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)
res_mdl, _ = mdl.multi_inter_by_name(
    ind, 
    model, 
    mdl.pop_israel, 
    [mdl.inter2name(ind, 100, no_risk=False)], 
    sim_length=500, 
    fix_vents=False,
    no_pop=False,
    start=len(date_lst),
)

model.reset()

TypeError: __init__() got an unexpected keyword argument 'theta'

In [12]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/mdl.pop_israel, res_mdl, date_lst, start=start,end=end)
# plt.vlines(x=14,ymin=0, ymax=0.0001)
# plt.vlines(x=23-6,ymin=0, ymax=0.0001, color='r')

AttributeError: module 'SEIR_full.calibration' has no attribute 'plot_calibrated_total_model'

In [ ]:
res_mdl['new_Is'].sum()

In [ ]:
model.noise

In [ ]:
mdl.plot_I_by_age(ind,res_mdl, new_only=True)

In [ ]:
mdl.plot_R_by_age(ind,res_mdl)

In [ ]:
t = np.arange(500)
plt.plot(t, 1 + np.cos((2 * np.pi * (t + model.phi)) / 365.0))

In [13]:
age_pct = []
for age in ind.A.values():
    age_pct.append(res_mdl['new_Is'][:, ind.age_dict[age]].sum()/res_mdl['new_Is'].sum())
plt.bar(list(ind.A.values()), age_pct)

NameError: name 'res_mdl' is not defined

# Model Calibration

## Scenario 1

### $\phi$=29

#### calibrating

In [ ]:
scen = 'Scenario1'
seasonality = True

In [ ]:
# fit params
start = pos_0303.index[0]
end = pos_0303.index[-1]
date_lst = pd.date_range('2020-02-20',end)
len(date_lst)

In [ ]:
model_1_ml = mdl.Model_behave(
    ind=ind,
    beta_j=np.array([0.10167117, 0.10167117, 0.10167117, 0.05606748, 0.05606748,
       0.04709613, 0.04709613, 0.25676354, 0.25676354])*0.4,
    theta=2.0684200685446243,
    beta_behave=0.38,
    scen=scen,
    seasonality=seasonality,
)

In [ ]:
# Model Fitting
p0=(0.04066847,0.02242699,0.01883845,0.10270542,2.0684200685446243, 0.38) # initial guess
bnds = ((0,0.3),(0,0.2),(0,0.2),(0,0.6),(1.5,3),(0,0.6))# boundries for variables


In [ ]:
model_1_ml.reset()

In [ ]:
res_fit_1_ml = model_1_ml.fit(
        mapper=region_age_250_to_30,
        p0=p0,
        bnds=bnds,
        data=data,
        date_lst = date_lst,
        start=start,
        end=end,
        loss_func='MSE',
        maxiter=85,
        factor=1,
    )

In [ ]:
mdl.print_stat_fit_behave(res_fit_1_ml)

In [ ]:
res_mdl_1_ml = model_1_ml.predict(
    C=mdl.C_calibration,
    days_in_season=len(date_lst),
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/mdl.pop_israel, res_mdl_1_ml, date_lst, start=start,end=end)
plt.vlines(x=14,ymin=0, ymax=0.0001)
plt.vlines(x=23-5,ymin=0, ymax=0.0001, color='r')

In [ ]:
#index to cut model's data
start_idx = int(np.where(date_lst == start)[0])
end_idx = int(np.where(date_lst == end)[0])

# creating DF
model_data_cal = res_mdl_1_ml['new_Is'][start_idx:end_idx + 1,:].sum(axis=1)
data_tot = (pd.DataFrame(data[1])/mdl.pop_israel).sum(axis=1)
plt.plot(range(len(data_tot)),data_tot, model_data_cal)
import scipy
scipy.stats.pearsonr(model_data_cal, data_tot)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(mdl_mapper=region_250_to_30,
                                          data=data, 
                                          mdl_data = res_mdl_1_ml['new_Is'],
                                          date_list = date_lst,
                                          start=start,
                                          end=end,
                                          region_name = {k:k for k in region_250_to_30.keys()},
                                          loss_func = 'MSE',
                                         ind=ind,
                                         data_mapper=region_ga_250_to_30)
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
age_cal = [
    ['0-4', '5-9', '10-19'],
    ['20-29', '30-39'],
    ['40-49', '50-59'],
    ['60-69', '70+'],
]
age_name = [
    '0-19', 
    '20-39', 
    '40-59', 
    '60+',
]
age_pct = []
age_pct_data = []
age_model = []
data_age = (pos_0303.sum(axis=0).unstack(level=0).sum(axis=1))/(pos_0303.sum(axis=0).unstack(level=0).sum(axis=1).sum())
for age_list in age_cal:
    idx = []
    for age in age_list:
        idx += list(ind.age_dict[age])
    age_pct.append(res_mdl_1_ml['new_Is'][:, idx].sum()/res_mdl_1_ml['new_Is'].sum())
    age_pct_data.append(data_age.loc[age_list].sum())

# set width of bar
barWidth = 0.25
 
# set height of bar
 
# Set position of bar on X axis
r1 = np.arange(len(age_pct))
r2 = [x - barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r2, age_pct, color='#557f2d', width=barWidth, edgecolor='white', label='model')
plt.bar(r3, age_pct_data, color='#2d7f5e', width=barWidth, edgecolor='white', label='data')
 
# Add xticks on the middle of the group bars
plt.xlabel('group', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(age_name))], age_name)
 
# Create legend & Show graphic
plt.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.plot(((res_mdl_1_ml['Vents']).sum(axis=1)[:(end-start).days])*isr_pop)
ax.set_ylabel('Resipratory cases [#]', fontsize=35)
ax.set_title('Respiratory Cases Global', fontsize=50)
ax.set_xlabel('Time [d]', fontsize=35)

In [ ]:
mdl.save_cal(res_fit_1_ml, ind, scen, 29)

#### running tracking calibration

In [ ]:
df = pd.read_csv('../Data/sick/daily_hospital_resp.csv')
df.index = pd.to_datetime(df['Date'], dayfirst=True)
resp = df['resp'].dropna()
hosp = df['hospitalized'].dropna()
# resp = pd.read_csv('../Data/sick/resp_country.csv')
# hosp = pd.read_csv('../Data/sick/hospitalizations_country.csv')

In [ ]:
ax= plt.subplot()
resp.plot(ax=ax)
hosp.plot(ax=ax)

In [ ]:
scen = 'Scenario1'
scen_idx = int(scen[-1])
seasonality = True
phase = 29
start_sim='2020-02-20'

start_date = hosp.index[0]
end_date = hosp.index[-1]

date_lst = pd.date_range(start_sim,end_date)
len(date_lst)

In [ ]:
# Model Fitting
p0=(1/7., 0.1) # initial guess
bnds = ((0.05,1.0),(0.05,0.25))# boundries for variables
# bnds = ((0.01,1.0),(0.01,1.0))# boundries for variables

In [ ]:
cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')

model_1_no = mdl.Model_behave(
    ind=ind,
    beta_j=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_j'],
    theta=cal_parameters[ind.cell_name][(scen_idx, phase)]['theta'],
    beta_behave=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_behave'],
    scen=scen,
    seasonality=seasonality,
)

In [ ]:
model_1_no.reset()
res_fit_1_no = model_1_no.fit_tracking(
        p0=p0,
        bnds=bnds,
        data=hosp.values,
        date_lst = date_lst,
        days_in_season=len(date_lst),
        start=start_date,
        end=end_date,
        loss_func='MSE',
        maxiter=300,
        factor=1,
        tracking='hosp'
    )

In [ ]:
mdl.print_stat_fit_hosp(res_fit_1_no)

In [ ]:
model_1_no.reset()
res_mdl_1_no = model_1_no.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_hospitalizations_calibration(res_mdl_1_no,hosp,date_lst,start_date=start_date,end_date=end_date)

In [ ]:
def save_cal_track(res_fit, ind, scen, phase, track):
    if track == 'H':
        cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['eta'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['nu'] = \
            res_fit.x[0]

    elif track == 'Vents':
        cal_parameters = pd.read_pickle(
            '../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['xi'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['mu'] = \
            res_fit.x[0]
    with open('../Data/calibration/calibrattion_dict.pickle', 'wb') as handle:
        pickle.dump(cal_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_cal_track(res_fit_1_no, ind, scen, phase, 'H')

In [ ]:
scen = 'Scenario1'
scen_idx = int(scen[-1])
seasonality = False
phase = '-'
start_sim='2020-02-20'

start_date = resp.index[0]
end_date = resp.index[-1]
date_lst = pd.date_range(start_sim,end_date)
len(date_lst)

In [ ]:
# Model Fitting
p0=(0.05, 1/9.9) # initial guess
# bnds = ((0.00,1),(1/14.,1/10.))# boundries for variables
bnds = ((0.00,1),(1/21.,1/10.))# boundries for variables

In [ ]:
cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')

model_1_no = mdl.Model_behave(
    ind=ind,
    beta_j=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_j'],
    theta=cal_parameters[ind.cell_name][(scen_idx, phase)]['theta'],
    beta_behave=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_behave'],
    scen=scen,
    seasonality=seasonality,
)

In [ ]:
model_1_no.reset()
res_fit_1_no = model_1_no.fit_tracking(
        p0=p0,
        bnds=bnds,
        data=resp.values,
        date_lst = date_lst,
        days_in_season=len(date_lst),
        start=start_date,
        end=end_date,
        loss_func='MSE',
        maxiter=85,
        factor=1,
        tracking='vents'
    )

In [ ]:
mdl.print_stat_fit_hosp(res_fit_1_no, tracking='vents')

In [ ]:
model_1_no.reset()
res_mdl_1_no = model_1_no.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_hospitalizations_calibration(res_mdl_1_no,resp.values,date_lst,start_date=start_date,end_date=end_date,
                                     tracking='vents')

In [ ]:
def save_cal_track(res_fit, ind, scen, phase, track):
    if track == 'H':
        cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['eta'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['nu'] = \
            res_fit.x[0]

    elif track == 'Vents':
        cal_parameters = pd.read_pickle(
            '../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['xi'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['mu'] = \
            res_fit.x[0]
    with open('../Data/calibration/calibrattion_dict.pickle', 'wb') as handle:
        pickle.dump(cal_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_cal_track(res_fit_1_no, ind, scen, phase, 'Vents')

#### Running Calibration forward with 100%

In [ ]:
model_1_ml.reset()
res_mdl = model_1_ml.predict(
    C=mdl.C_calibration,
    days_in_season=len(date_lst),
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)
res_mdl, _ = mdl.multi_inter_by_name(
    ind, 
    model_1_ml, 
    mdl.pop_israel, 
    [mdl.inter2name(ind, 100, no_risk=True)], 
    sim_length=500, 
    fix_vents=False,
    no_pop=False,
    start=len(date_lst),
)

In [ ]:
mdl.plot_I_by_age(ind,res_mdl, new_only=True)

In [ ]:
# R = res_mdl['Vents']*isr_pop
R = res_mdl['R']

# dictionary of arrays to plot
plot_dict ={}
plot_dict = R.sum(axis=1)

fig = plt.figure(figsize=(15,10))
ax = plt.subplot()
ax.set_ylabel('Total Recovered cases [#]', fontsize=35)
ax.set_title('Recovered Cases Global by Age', fontsize=50)
ax.set_xlabel('Time [d]', fontsize=35)
plot_df = pd.DataFrame.from_dict(plot_dict)

# plot
plot_df.plot(ax=ax)
plt.show()
plt.close()
return fig, ax

In [ ]:
t = np.arange(500)
plt.plot(t, 1 + np.cos((2 * np.pi * (t + model_1_ml.phi)) / 365.0))

### NO seasonality

#### calibration

In [ ]:
scen = 'Scenario1'
seasonality = False

In [ ]:
# fit params
start = pos_0303.index[0]
end = pos_0303.index[-1]
date_lst = pd.date_range('2020-02-20',end)
len(date_lst)

In [ ]:
model_1_ml = mdl.Model_behave(
    ind=ind,
    beta_j=np.array([0.10167117, 0.10167117, 0.10167117, 0.05606748, 0.05606748,
       0.04709613, 0.04709613, 0.25676354, 0.25676354])*0.4,
    theta=2.0684200685446243,
    beta_behave=0.38,
    scen=scen,
    seasonality=seasonality,
)

In [ ]:
# Model Fitting
p0=(0.04066847,0.02242699,0.01883845,0.10270542,2.0684200685446243, 0.38) # initial guess
bnds = ((0,0.3),(0,0.2),(0,0.2),(0,0.6),(1.5,3),(0,0.6))# boundries for variables


In [ ]:
res_fit_1_ml = model_1_ml.fit(
        mapper=region_age_250_to_30,
        p0=p0,
        bnds=bnds,
        data=data,
        date_lst = date_lst,
        start=start,
        end=end,
        loss_func='MSE',
        maxiter=85,
        factor=1,
    )

In [ ]:
res_fit_1_ml.x

In [ ]:
mdl.print_stat_fit_behave(res_fit_1_ml)

In [ ]:
mdl.save_cal(res_fit_1_ml, ind, scen, '-')

In [ ]:
res_mdl_1_ml = model_1_ml.predict(
    C=mdl.C_calibration,
    days_in_season=len(date_lst),
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/mdl.pop_israel, res_mdl_1_ml, date_lst, start=start,end=end)
plt.vlines(x=14,ymin=0, ymax=0.0001)
plt.vlines(x=23-5,ymin=0, ymax=0.0001, color='r')

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(mdl_mapper=region_250_to_30,
                                          data=data, 
                                          mdl_data = res_mdl_1_ml['new_Is'],
                                          date_list = date_lst,
                                          start=start,
                                          end=end,
                                          region_name = {k:k for k in region_250_to_30.keys()},
                                          loss_func = 'MSE',
                                         ind=ind,
                                         data_mapper=region_ga_250_to_30)
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
age_cal = [
    ['0-4', '5-9', '10-19'],
    ['20-29', '30-39'],
    ['40-49', '50-59'],
    ['60-69', '70+'],
]
age_name = [
    '0-19', 
    '20-39', 
    '40-59', 
    '60+',
]
age_pct = []
age_pct_data = []
age_model = []
data_age = (pos_0303.sum(axis=0).unstack(level=0).sum(axis=1))/(pos_0303.sum(axis=0).unstack(level=0).sum(axis=1).sum())
for age_list in age_cal:
    idx = []
    for age in age_list:
        idx += list(ind.age_dict[age])
    age_pct.append(res_mdl_1_ml['new_Is'][:, idx].sum()/res_mdl_1_ml['new_Is'].sum())
    age_pct_data.append(data_age.loc[age_list].sum())

# set width of bar
barWidth = 0.25
 
# set height of bar
 
# Set position of bar on X axis
r1 = np.arange(len(age_pct))
r2 = [x - barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r2, age_pct, color='#557f2d', width=barWidth, edgecolor='white', label='model')
plt.bar(r3, age_pct_data, color='#2d7f5e', width=barWidth, edgecolor='white', label='data')
 
# Add xticks on the middle of the group bars
plt.xlabel('group', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(age_name))], age_name)
 
# Create legend & Show graphic
plt.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.plot(((res_mdl_1_ml['Vents']).sum(axis=1)[:(end-start).days])*isr_pop)
ax.set_ylabel('Resipratory cases [#]', fontsize=35)
ax.set_title('Respiratory Cases Global', fontsize=50)
ax.set_xlabel('Time [d]', fontsize=35)

#### running tracking calibration 

In [ ]:
df = pd.read_csv('../Data/sick/daily_hospital_resp.csv')
df.index = pd.to_datetime(df['Date'], dayfirst=True)
resp = df['resp'].dropna()
hosp = df['hospitalized'].dropna()
# resp = pd.read_csv('../Data/sick/resp_country.csv')
# hosp = pd.read_csv('../Data/sick/hospitalizations_country.csv')

In [ ]:
ax= plt.subplot()
resp.plot(ax=ax)
hosp.plot(ax=ax)

In [ ]:
scen = 'Scenario1'
scen_idx = int(scen[-1])
seasonality = False
phase = '-'
start_sim='2020-02-20'

start_date = hosp.index[0]
end_date = hosp.index[-1]

date_lst = pd.date_range(start_sim,end_date)
len(date_lst)

In [ ]:
hosp.index[-1]

In [ ]:
# Model Fitting
p0=(1/7., 0.1) # initial guess
bnds = ((0.05,1.0),(0.05,0.25))# boundries for variables
# bnds = ((0.01,1.0),(0.01,1.0))# boundries for variables

In [ ]:
cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')

model_1_no = mdl.Model_behave(
    ind=ind,
    beta_j=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_j'],
    theta=cal_parameters[ind.cell_name][(scen_idx, phase)]['theta'],
    beta_behave=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_behave'],
    scen=scen,
    seasonality=seasonality,
)

In [ ]:
model_1_no.reset()
res_fit_1_no = model_1_no.fit_tracking(
        p0=p0,
        bnds=bnds,
        data=hosp.values,
        date_lst = date_lst,
        days_in_season=len(date_lst),
        start=start_date,
        end=end_date,
        loss_func='MSE',
        maxiter=300,
        factor=1,
        tracking='hosp'
    )

In [ ]:
mdl.print_stat_fit_hosp(res_fit_1_no)

In [ ]:
model_1_no.reset()
res_mdl_1_no = model_1_no.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_hospitalizations_calibration(res_mdl_1_no,hosp,date_lst,start_date=start_date,end_date=end_date)

In [ ]:
def save_cal_track(res_fit, ind, scen, phase, track):
    if track == 'H':
        cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['eta'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['nu'] = \
            res_fit.x[0]

    elif track == 'Vents':
        cal_parameters = pd.read_pickle(
            '../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['xi'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['mu'] = \
            res_fit.x[0]
    with open('../Data/calibration/calibrattion_dict.pickle', 'wb') as handle:
        pickle.dump(cal_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_cal_track(res_fit_1_no, ind, scen, phase, 'H')

In [ ]:
scen = 'Scenario1'
scen_idx = int(scen[-1])
seasonality = False
phase = '-'
start_sim='2020-02-20'

start_date = resp.index[0]
end_date = resp.index[-1]
date_lst = pd.date_range(start_sim,end_date)
len(date_lst)

In [ ]:
resp.index[-1]

In [ ]:
# Model Fitting
p0=(0.05, 1/9.9) # initial guess
# bnds = ((0.00,1),(1/14.,1/10.))# boundries for variables
bnds = ((0.00,1),(1/21.,1/10.))# boundries for variables

In [ ]:
cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')

model_1_no = mdl.Model_behave(
    ind=ind,
    beta_j=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_j'],
    theta=cal_parameters[ind.cell_name][(scen_idx, phase)]['theta'],
    beta_behave=cal_parameters[ind.cell_name][(scen_idx, phase)]['beta_behave'],
    scen=scen,
    seasonality=seasonality,
)

In [ ]:
model_1_no.reset()
res_fit_1_no = model_1_no.fit_tracking(
        p0=p0,
        bnds=bnds,
        data=resp.values,
        date_lst = date_lst,
        days_in_season=len(date_lst),
        start=start_date,
        end=end_date,
        loss_func='MSE',
        maxiter=85,
        factor=1,
        tracking='vents'
    )

In [ ]:
mdl.print_stat_fit_hosp(res_fit_1_no, tracking='vents')

In [ ]:
model_1_no.reset()
res_mdl_1_no = model_1_no.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_hospitalizations_calibration(res_mdl_1_no,resp.values,date_lst,start_date=start_date,end_date=end_date,
                                     tracking='vents')

In [ ]:
def save_cal_track(res_fit, ind, scen, phase, track):
    if track == 'H':
        cal_parameters = pd.read_pickle('../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['eta'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['nu'] = \
            res_fit.x[0]

    elif track == 'Vents':
        cal_parameters = pd.read_pickle(
            '../Data/calibration/calibrattion_dict.pickle')
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['xi'] = \
            res_fit.x[0]
        cal_parameters[ind.cell_name][(int(scen[-1]), phase)]['mu'] = \
            res_fit.x[0]
    with open('../Data/calibration/calibrattion_dict.pickle', 'wb') as handle:
        pickle.dump(cal_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_cal_track(res_fit_1_no, ind, scen, phase, 'Vents')

#### running calibration with 100%

In [ ]:
model_1_ml.reset()
res_mdl = model_1_ml.predict(
    C=mdl.C_calibration,
    days_in_season=len(date_lst),
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)
res_mdl, _ = mdl.multi_inter_by_name(
    ind, 
    model_1_ml, 
    mdl.pop_israel, 
    [mdl.inter2name(ind, 100, no_risk=False)], 
    sim_length=500, 
    fix_vents=False,
    no_pop=False,
    start=len(date_lst),
)

In [ ]:
mdl.plot_I_by_age(ind,res_mdl, new_only=True)

In [ ]:
R = res_mdl['Vents']*isr_pop
# R = res_mdl['R']

# dictionary of arrays to plot
plot_dict ={}
plot_dict = R.sum(axis=1)

fig = plt.figure(figsize=(15,10))
ax = plt.subplot()
ax.set_ylabel('Total Recovered cases [#]', fontsize=35)
ax.set_title('Recovered Cases Global by Age', fontsize=50)
ax.set_xlabel('Time [d]', fontsize=35)
plot_df = pd.DataFrame.from_dict(plot_dict)

# plot
plot_df.plot(ax=ax)
plt.show()
plt.close()
print('total casulties: ', mdl.make_casulties(res_mdl, -1, mdl.pop_israel, model_1_ml.mu))

In [ ]:
t = np.arange(500)
plt.plot(t, 1 + np.cos((2 * np.pi * (t + model_1_ml.phi)) / 365.0))

# Fit model Matan

## Scenario 2

In [ ]:
# fit params
days_in_season=100
date_lst = pd.date_range('2020-02-20',last)
len(date_lst)

In [ ]:
# Model Fitting
p0=(0.0981001,0.05305112,0.05157651,0.3,2.103427546494595, 0.23785134529029744) # initial guess
bnds = ((0,0.1),(0,0.15),(0,0.15),(0,0.4),(1.5,3),(0,0.6))# boundries for variables


In [ ]:
model.reset()
model.update({'sigma':1./4.4, 'nu':0.1})

In [ ]:
res_fit = model.fit(
        p0=p0,
        bnds=bnds,
        data=data,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=300,
        factor=1,
    )

In [ ]:
mdl.print_stat_fit_behave(res_fit)

In [ ]:
model.reset()
model.update({'nu':1/15.})
res_mdl = model.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
model.beta_j

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data, 
                                          mdl_data = res_mdl['new_Is'],
                                          date_list = date_lst,
                                          start='2020-02-29',
                                          end='2020-04-12',
                                          region_name = mdl.cell,
                                          loss_func = 'MSE')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
(res_mdl['new_Is']).sum()

In [ ]:
plt.plot(res_mdl['H'].sum(axis=1)*9136000)

In [ ]:
date_lst[-1]

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/9136000,res_mdl,date_lst,start='2020-02-26',end='2020-04-09')

### Model Variance

In [ ]:
model_start_idx = int(np.where(date_lst == '2020-02-26')[0])
model_end_idx = int(np.where(date_lst == '2020-04-09')[0])

In [ ]:
# Calculation of variance:
# model results array based on region and age:
model_by_age_region = {}
for i, key in enumerate(mdl.region_age_dict.keys()):
    model_by_age_region[key[0] + '_' + key[1] + '_mdl'] = res_mdl['new_Is'][model_start_idx:model_end_idx+1, mdl.region_age_dict[key]].sum(axis=1)
model_by_age_region = pd.DataFrame.from_dict(model_by_age_region)

In [ ]:
model_std = (np.sqrt((np.power((data[1]/isr_pop - model_by_age_region.values),2).sum())/data[1].size))

In [ ]:
model_std

### Fit UB for different CIs

In [ ]:
all_calibrations = {}
for percent,z in [('80%',0.842), ('75%',0.674), ('70%',0.524)]:
    res_model = {}
    print('======checking CI {}%======='.format(100 - (100-float(percent[:-1]))/2))
    # calculate new data:
    new_data = data.copy()
    new_data[1] = ((data[1]/isr_pop) + z*model_std)*isr_pop
    
    # initializing model:
    curr_mdl = mdl.Model_behave(
    beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245]),
    theta=2.826729434860104,
    beta_behave=0.5552998605894367,
    eps=mdl.eps_sector['Scenario2'],
    f=mdl.f0_full['Scenario2'],
    sigma=1/4.4,
    nu=0.1
    )
#      Model Fitting
    days_in_season=70
    p0=(0.0981001,0.05305112,0.05157651,0.3,2.103427546494595, 0.23785134529029744) # initial guess
    bnds = ((0,0.15),(0,0.15),(0,0.15),(0,0.54),(1.5,3),(0,0.7))# boundries for variables
    
    res_fit_curr = curr_mdl.fit(
        p0=p0,
        bnds=bnds,
        data=new_data,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=300,
        factor=1,
    )
    mdl.print_stat_fit_behave(res_fit_curr)
    
    curr_mdl.reset()
    res_mdl_curr = curr_mdl.predict(
        C=mdl.C_calibration,
        days_in_season=200,
        stay_home_idx=mdl.stay_home_idx,
        not_routine=mdl.not_routine,
    )
        
        # plotting the data and model
    fig,ax = mdl.plot_calibrated_model_region(data=new_data, 
                                              mdl_data = res_mdl_curr['new_Is'],
                                              date_list = date_lst,
                                              start='2020-02-26',
                                              end='2020-04-09',
                                              region_name = mdl.cell,
                                              loss_func = 'MSE')
    fig.set_size_inches((30,30))
    plt.tight_layout()
    plt.show()
    
    all_calibrations[percent] = {'beta_j':curr_mdl.beta_j, 'beta_behave':curr_mdl.beta_behave, 'theta':curr_mdl.theta}
    
    

In [ ]:
# Save calibrated params
with open('calibrattion_dict.pickle', 'wb') as handle:
    pickle.dump(all_calibrations, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Fit UB

In [ ]:
# UB data:
data_ub = data.copy()
data_ub[1] = ((data[1]/isr_pop) + 1.96*model_std)*isr_pop

In [ ]:
model_ub = mdl.Model_behave(
    beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245]),
    theta=2.826729434860104,
    beta_behave=0.5552998605894367,
    eps=mdl.eps_sector['Scenario2'],
    f=mdl.f0_full['Scenario2'],
    sigma=1/4.4,
    nu=0.1
)

In [ ]:
# Model Fitting
days_in_season=70
p0=(0.0981001,0.05305112,0.05157651,0.3,2.103427546494595, 0.23785134529029744) # initial guess
bnds = ((0,0.15),(0,0.15),(0,0.15),(0,0.54),(1.5,3),(0,0.7))# boundries for variables

In [ ]:
res_fit_ub = model_ub.fit(
        p0=p0,
        bnds=bnds,
        data=data_ub,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=300,
        factor=1,
    )

In [ ]:
mdl.print_stat_fit_behave(res_fit_ub)

In [ ]:
model_ub.reset()
res_mdl_ub = model_ub.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data_ub, 
                                          mdl_data = res_mdl_ub['new_Is'],
                                          date_list = date_lst,
                                          start='2020-02-29',
                                          end='2020-04-12',
                                          region_name = mdl.cell,
                                          loss_func = 'MSE')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl_ub['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
plt.plot(res_mdl_ub['H'].sum(axis=1)*9136000)

In [ ]:
res_mdl_ub['new_Is'].sum()

In [ ]:
res_mdl['new_Is'][:55].sum(axis=1).sum()*9136000

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data_ub[1])/9136000,res_mdl_ub,date_lst,start='2020-02-26',end='2020-04-09')

### Fit LB

In [ ]:
# LB data:
data_lb = data.copy()
data_lb[1] = ((data[1]/9136000)*(1/3.))*isr_pop # beacuse the data_ub is 3 times bigger than data (on average)

In [ ]:
model_lb = mdl.Model_behave(
    beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245]),
    theta=2.826729434860104,
    beta_behave=0.5552998605894367,
    eps=mdl.eps_sector['Scenario2'],
    f=mdl.f0_full['Scenario2'],
    sigma=1/4.4,
    nu = 0.1
)

In [ ]:
# Model Fitting
days_in_season=70
p0=(0.0981001,0.05305112,0.05157651,0.3,2.103427546494595, 0.23785134529029744) # initial guess
bnds = ((0,0.15),(0,0.15),(0,0.15),(0,0.54),(1.5,3),(0,0.7))# boundries for variables

In [ ]:
res_fit_lb = model_lb.fit(
        p0=p0,
        bnds=bnds,
        data=data_lb,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=300,
        factor=1,
    )

In [ ]:
mdl.print_stat_fit_behave(res_fit_lb)

In [ ]:
model_lb.reset()
res_mdl_lb = model_lb.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data_lb, 
                                          mdl_data = res_mdl_ub_lb['new_Is'],
                                          date_list = date_lst,
                                          start='2020-02-29',
                                          end='2020-04-12',
                                          region_name = mdl.cell,
                                          loss_func = 'MSE')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl_lb['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
plt.plot(res_mdl_lb['H'].sum(axis=1)*9136000)

In [ ]:
res_mdl_lb['new_Is'][:55].sum(axis=1).sum()

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data_lb[1])/9136000,res_mdl_lb,date_lst,start='2020-02-26',end='2020-04-09')

## Scenario 1

In [ ]:
scen = 'Scenario1'
sc1_res_dict = {}

In [ ]:
model_1_ml = mdl.Model_behave(
    ind,
    beta_j=np.array([0.02184738 ,0.02184738 ,0.02184738 ,0.00211441,0.00211441,0.0,0.0,0.0,0.0]),
    theta=3.0,
    beta_behave=0.6,
    eps=mdl.eps_sector[scen],
    f=mdl.f0_full[scen],
    seasonality=True
                              
)

In [ ]:
# fit params
start = pos_0303.index[0]
end = pos_0303.index[-1]
date_lst = pd.date_range('2020-02-21',end)
len(date_lst)

In [ ]:
# Model Fitting
p0=(0.0981001,0.05305112,0.05157651,0.3,2.103427546494595, 0.23785134529029744) # initial guess
bnds = ((0,0.3),(0,0.2),(0,0.2),(0,0.6),(1.5,3),(0,0.6))# boundries for variables


In [ ]:
model_1_ml.reset()

In [ ]:
res_fit_1_ml = model_1_ml.fit(
        mapper=region_age_250_to_30,
        p0=p0,
        bnds=bnds,
        data=data,
        date_lst = date_lst,
        start=start,
        end=end,
        loss_func='MSE',
        maxiter=85,
        factor=1,
    )

In [ ]:
res_fit_1_ml.x[5]

In [ ]:
print('minimized value:', res_fit_1_ml.fun)
print('Fitted parameters:\n Beta={0}\n Theta={1},\n,Beta_behave={2}'.format(res_fit_1_ml.x[:4],
                                                                                  res_fit_1_ml.x[4],
                                                                                  res_fit_1_ml.x[5],))

print('num of sampling the target function:', res_fit_1_ml.nfev)

In [ ]:
mdl.print_stat_fit_behave(res_fit_1_ml)

In [ ]:
# model_1_ml.reset()
model_1_ml.update({'beta_behave': 0.01})
res_mdl_1_ml = model_1_ml.predict(
    C=mdl.C_calibration,
    days_in_season=100,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:

print(model_1_ml.theta)
print(model_1_ml.beta_behave)
model_1_ml.beta_j

In [ ]:
model_1_ml.sigma

In [ ]:
model_1_ml.update({'seasonality':0.0,'noise':0.0, 'beta_j':np.array([0.10167117, 0.10167117, 0.10167117, 0.05606748, 0.05606748,
       0.04709613, 0.04709613, 0.25676354, 0.25676354])*0.95,'beta_behave':0.18876288297385702*1.45,'theta':2.0684200685446243,
                  'theta_arab':1})

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(mdl_mapper=region_250_to_30,
                                          data=data, 
                                          mdl_data = res_mdl['new_Is'],
                                          date_list = date_lst,
                                          start=start,
                                          end=end,
                                          region_name = {k:k for k in region_250_to_30.keys()},
                                          loss_func = 'MSE',
                                         ind=ind,
                                         data_mapper=region_ga_250_to_30)
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/mdl.pop_israel, res_mdl, date_lst, start=start,end=end)
# plt.vlines(x=14,ymin=0, ymax=0.0001)
# plt.vlines(x=23-6,ymin=0, ymax=0.0001, color='r')

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(mdl_mapper=region_250_to_30,
                                          data=data, 
                                          mdl_data = res_mdl_1_ml['new_Is'],
                                          date_list = date_lst,
                                          start=start,
                                          end=end,
                                          region_name = {k:k for k in region_250_to_30.keys()},
                                          loss_func = 'MSE',
                                         ind=ind,
                                         data_mapper=region_ga_250_to_30)
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
pos['11_haredi'].sum(axis=1).sum()

In [ ]:
data[1].shape

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl_1_ml['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
(res_mdl_1_ml['new_Is']).sum()

In [ ]:
plt.plot(res_mdl_1_ml['Vents'].sum(axis=1)*9136000)

In [ ]:
date_lst[-1]

In [ ]:
res_fit_1_ml

In [ ]:
check = res_mdl_1_ml['S']+res_mdl_1_ml['E']+res_mdl_1_ml['Ia']+res_mdl_1_ml['Ie']+res_mdl_1_ml['Is']+res_mdl_1_ml['R']

In [ ]:
plt.plot(check.sum(axis=1)-1)
plt.vlines(x=18, ymin=-0.0,ymax=0.0002, linestyles='dashed')

In [ ]:
check2=res_mdl_1_ml['L'][:]# + res_mdl_1_ml['E'][:]
plt.plot(check2.sum(axis=1)[:50])
# plt.vlines(x=18, ymin=-0.0,ymax=0.0007, linestyles='dashed')

In [ ]:
count = 0
for a in model_1_ml.eps[:18]:
    count += a.sum()


In [ ]:
model_1_ml.eps

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/mdl.pop_israel,res_mdl_1_ml,date_lst,start='2020-02-26',end='2020-04-23')
# plt.vlines(x=14,ymin=0, ymax=0.0001)
# plt.vlines(x=23-6,ymin=0, ymax=0.0001, color='r')

In [ ]:
# Saving new params:
sc1_res_dict['ml'] = {'beta_j':model_1_ml.beta_j,
                  'theta': model_1_ml.theta,
                  'beta_behave': model_1_ml.beta_behave}

In [ ]:
int(np.where(date_lst == '2020-03-14')[0])

In [ ]:
mdl.plot_I_by_age(cell_250,res_mdl_1_ml)

In [ ]:
res_mdl_1_ml['R'][-1,:].sum()

In [ ]:
mdl_1_ml.sigma

### Model Variance

In [ ]:
model_start_idx = int(np.where(date_lst == '2020-02-26')[0])
model_end_idx = int(np.where(date_lst == '2020-04-09')[0])

In [ ]:
# Calculation of variance:
# model results array based on region and age:
model_by_age_region = {}
for i, key in enumerate(mdl.region_age_dict.keys()):
    model_by_age_region[key[0] + '_' + key[1] + '_mdl'] = res_mdl_1_ml['new_Is'][model_start_idx:model_end_idx+1, mdl.region_age_dict[key]].sum(axis=1)
model_by_age_region = pd.DataFrame.from_dict(model_by_age_region)

In [ ]:
model_std = (np.sqrt((np.power((data[1]/mdl.pop_israel - model_by_age_region.values),2).sum())/data[1].size))

In [ ]:
model_std

### Fit UB

In [ ]:
# UB data:
data_ub = data.copy()
data_ub[1] = ((data[1]/mdl.pop_israel) + 1.96*model_std)*mdl.pop_israel

In [ ]:
model_sc1_ub = mdl.Model_behave(
    beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245]),
    theta=2.826729434860104,
    beta_behave=0.5552998605894367,
    eps=mdl.eps_sector['Scenario1'],
    f=mdl.f0_full['Scenario1'],
)

In [ ]:
# Model Fitting
days_in_season=70
p0=(0.15,0.07317271 ,0.09578334 ,0.21491134,1.5, 0.34196516) # initial guess
bnds = ((0,0.2),(0,0.15),(0,0.15),(0,0.54),(1.0,3),(0,0.7))# boundries for variables

In [ ]:
res_fit_sc1_ub = model_sc1_ub.fit(
        p0=p0,
        bnds=bnds,
        data=data_ub,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=400,
        factor=1,
    )

In [ ]:
mdl.print_stat_fit_behave(res_fit_sc1_ub)

In [ ]:
model_sc1_ub.reset()
res_mdl_sc1_ub = model_sc1_ub.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data, 
                                          mdl_data = res_mdl_sc1_ub['new_Is'],
                                          date_list = date_lst,
                                          start='2020-02-29',
                                          end='2020-04-12',
                                          region_name = mdl.cell,
                                          loss_func = 'MSE')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl_sc1_ub['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
plt.plot(res_mdl_sc1_ub['Vents'].sum(axis=1)*mdl.pop_israel)

In [ ]:
res_mdl_sc1_ub['new_Is'].sum()

In [ ]:
res_mdl['new_Is'][:55].sum(axis=1).sum()*9136000

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data[1])/mdl.pop_israel,res_mdl_sc1_ub,date_lst,start='2020-02-26',end='2020-04-09')

In [ ]:
# Saving new params:
sc1_res_dict['ub'] = {'beta_j':model_sc1_ub.beta_j,
                  'theta': model_sc1_ub.theta,
                  'beta_behave': model_sc1_ub.beta_behave}

### Fit LB

In [ ]:
# LB data:
data_lb = data.copy()
data_lb[1] = ((data[1]/mdl.pop_israel)*(1/3.))*mdl.pop_israel # beacuse the data_ub is 3 times bigger than data (on average)

In [ ]:
model_sc1_lb = mdl.Model_behave(
    beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245]),
    theta=2.826729434860104,
    beta_behave=0.5552998605894367,
    eps=mdl.eps_sector['Scenario1'],
    f=mdl.f0_full['Scenario1'],
)

In [ ]:
# Model Fitting
days_in_season=70
p0=(0.15,0.07317271 ,0.09578334 ,0.21491134,1.5, 0.34196516) # initial guess
bnds = ((0,0.2),(0,0.15),(0,0.15),(0,0.54),(1.0,3),(0,0.7))# boundries for variables

In [ ]:
res_fit_sc1_lb = model_sc1_lb.fit(
        p0=p0,
        bnds=bnds,
        data=data_lb,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=400,
        factor=1,
    )

In [ ]:
mdl.print_stat_fit_behave(res_fit_sc1_lb)

In [ ]:
model_sc1_lb.reset()
res_mdl_sc1_lb = model_sc1_lb.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data_lb, 
                                          mdl_data = res_mdl_sc1_lb['new_Is'],
                                          date_list = date_lst,
                                          start='2020-02-29',
                                          end='2020-04-12',
                                          region_name = mdl.cell,
                                          loss_func = 'MSE')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl_sc1_lb['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
plt.plot(res_mdl_sc1_lb['Vents'].sum(axis=1)*9136000)

In [ ]:
res_mdl_sc1_lb['new_Is'].sum()

In [ ]:
mdl.plot_calibrated_total_model(pd.DataFrame(data_lb[1])/mdl.pop_israel,res_mdl_sc1_lb,date_lst,start='2020-02-26',end='2020-04-09')

In [ ]:
# Saving new params:
sc1_res_dict['lb'] = {'beta_j':model_sc1_lb.beta_j,
                  'theta': model_sc1_lb.theta,
                  'beta_behave': model_sc1_lb.beta_behave}

### Fit UB for different CIs

In [ ]:
all_calibrations = {}
for percent,z in [('80%',0.842), ('75%',0.674), ('70%',0.524)]:
    res_model = {}
    print('======checking CI {}%======='.format(100 - (100-float(percent[:-1]))/2))
    # calculate new data:
    new_data = data.copy()
    new_data[1] = ((data[1]/mdl.pop_israel) + z*model_std)*mdl.pop_israel
    
    # initializing model:
    curr_mdl = mdl.Model_behave(
    beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245]),
    theta=2.826729434860104,
    beta_behave=0.5552998605894367,
    eps=mdl.eps_sector['Scenario1'],
    f=mdl.f0_full['Scenario1'],
    sigma=1/4.4,
    nu=0.1
    )
#      Model Fitting
    days_in_season=70
    p0=(0.15,0.07317271 ,0.09578334 ,0.21491134,1.5, 0.34196516) # initial guess
    bnds = ((0,0.2),(0,0.15),(0,0.15),(0,0.54),(1.0,3),(0,0.7))# boundries for variabless
    
    res_fit_curr = curr_mdl.fit(
        p0=p0,
        bnds=bnds,
        data=new_data,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start='2020-02-26',
        end='2020-04-09',
        loss_func='MSE',
        maxiter=300,
        factor=1,
    )
    mdl.print_stat_fit_behave(res_fit_curr)
    
    curr_mdl.reset()
    res_mdl_curr = curr_mdl.predict(
        C=mdl.C_calibration,
        days_in_season=200,
        stay_home_idx=mdl.stay_home_idx,
        not_routine=mdl.not_routine,
    )
        
        # plotting the data and model
    fig,ax = mdl.plot_calibrated_model_region(data=new_data, 
                                              mdl_data = res_mdl_curr['new_Is'],
                                              date_list = date_lst,
                                              start='2020-02-26',
                                              end='2020-04-09',
                                              region_name = mdl.cell,
                                              loss_func = 'MSE')
    fig.set_size_inches((30,30))
    plt.tight_layout()
    plt.show()
    
    all_calibrations[percent] = {'beta_j':curr_mdl.beta_j, 'beta_behave':curr_mdl.beta_behave, 'theta':curr_mdl.theta}
    
    

In [ ]:
sc1_res_dict.update(all_calibrations)

In [ ]:
sc1_res_dict

In [ ]:
# Save calibrated params
with open('../Data/calibration/calibration_sc1_dict.pickle', 'wb') as handle:
    pickle.dump(sc1_res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Amit's fit

## Data old

In [ ]:
# data = pd.read_csv('../Data/sick/old_data2003_1304.csv', index_col=0)
# data.head()

## Scenario 2

In [ ]:
# fit params
days_in_season=70
date_lst = pd.date_range('2020-02-20',last)
len(date_lst)

In [ ]:
# Model Fitting
p0=(0.03574392,0.0680812,0.07149694,0.24919698,1.8512674898217287, 0.40194272460656777) # initial guess
bnds = ((0,0.1),(0,0.15),(0,0.15),(0,0.3),(1.5,3),(0,0.6))# boundries for variables
res_fit = []
fitted_params = []
fitted_beta = []
theta = []
######## Doind Exp:
######## checking shift, different losses, and factors:
for i, (factor, shift, loss) in enumerate(itertools.product(
    [1 ,10 ,100, 500, 1000],
    np.arange(-7, 7, 1),
    ['MSE', 'BIN', 'POIS', 'POIS_NAIV'],
)):
    print('Checking combination: ', (factor, shift, loss))
    #defining shift
    pos_shift = pos.copy()
    tot_shift = tot.copy()
    pos_shift.index = pd.DatetimeIndex(pos.index + pd.DateOffset(shift))
    tot_shift.index = pd.DatetimeIndex(tot.index + pd.DateOffset(shift))
    pos_shift.index = pd.to_datetime(pos_shift.index)
    tot_shift.index = pd.to_datetime(tot_shift.index)
    min_d = max([pos_shift.index.values.min(), first])
    max_d = min([pos_shift.index.values.max(), last])
    pos_shift = pos_shift.loc[pd.date_range(min_d,max_d)]
    tot_shift = tot_shift.loc[pd.date_range(min_d,max_d)]
    
    data = np.ndarray((2,len(pos_shift), len(mdl.GA.values())))
    data[0] = tot_shift
    data[1] = pos_shift
    
    res_fit.append(model.fit(
        p0=p0,
        bnds=bnds,
        data=data,
        date_lst = date_lst,
        days_in_season=days_in_season,
        start=min_d,
        end=max_d,
        loss_func=loss,
        maxiter=300,
        factor=factor,
    ))
    # errorfunc(tpl,data,f,season_length, eps, alpha)


    fitted_params.append(res_fit.x)

    # run the fitted model:
    fitted_beta.append(np.array([fitted_params[0], fitted_params[0], fitted_params[0], fitted_params[1], fitted_params[1],\
                            fitted_params[2], fitted_params[2], fitted_params[3], fitted_params[3]]))
    theta.append(fitted_params[4])
    beta_behave.append(fitted_params[5])
                        
# print data stat
mdl.print_stat_fit_behave(res_fit)


In [ ]:
# # last
# fitted_beta = [0.08401885, 0.0498636, 0.04742293, 0.29044575]
# fitted_beta = np.array([fitted_beta[0], fitted_beta[0], fitted_beta[0], 
#                    fitted_beta[1], fitted_beta[1], 
#                    fitted_beta[2], fitted_beta[2], 
#                    fitted_beta[3], fitted_beta[3]])
# theta = 1.7884756534064268
# beta_behave = 0.3114150965784499

In [ ]:
# model.update({
#     'beta_j': fitted_beta,
#     'theta': theta,
#     'beta_behave': beta_behave,
# })

In [ ]:
model.reset()
res_mdl = model.predict(
    C=mdl.C_calibration,
    days_in_season=200,
    stay_home_idx=mdl.stay_home_idx,
    not_routine=mdl.not_routine,
)

In [ ]:
mdl.plot_I_by_age(res_mdl)

In [ ]:
res_mdl['new_Is'].sum()

In [ ]:
plt.plot((res_mdl['L'].sum(axis=1)))

In [ ]:
plt.plot((res_mdl['H'].sum(axis=1))*9136000)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data, 
                                          mdl_data = res_mdl['new_Is'],
                                          date_list = date_lst,
                                          start=first,
                                          end=last,
                                          region_name = mdl.cell,
                                          loss_func = 'Pois_Naiv')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.cell[region]] = (res_mdl['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

## old fit results

In [ ]:
# best old fit results:
# alpha=1
# beta_j=np.array([0.02927922 ,0.02927922 ,0.02927922 ,0.04655266,0.04655266,0.05775265,0.05775265,0.18444245,0.18444245])
# theta=2.826729434860104
# beta_behave=0.5552998605894367
fitted_beta = [0.08401885, 0.0498636, 0.04742293, 0.29044575]
fitted_beta = np.array([beta_j[0], beta_j[0], beta_j[0], 
                   beta_j[1], beta_j[1], 
                   beta_j[2], beta_j[2], 
                   beta_j[3], beta_j[3]])
theta = 1.7884756534064268
beta_behave = 0.3114150965784499

In [ ]:
res_mdl = mdl.run_sector_model_behave(beta_home=0.38/9., theta=theta, beta_j=beta_j, beta_behave=beta_behave,
                                        eps=mdl.eps_sector['Scenario2'], f=mdl.f0_full['Scenario2'], alpha=alpha, 
                                        days_in_season=175, nu=1/10.)

In [ ]:
mdl.plot_I_by_age(res_mdl)

In [ ]:
res_mdl['new_Is'].sum()

In [ ]:
plt.plot((res_mdl['L'].sum(axis=1)))

In [ ]:
plt.plot((res_mdl['H'].sum(axis=1))*9136000)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=data.values, mdl_data = res_mdl['new_Is'],date_list = date_lst,
                                         start='2020-03-12',end='2020-04-05')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
mdl.plot_calibrated_total_model(data=data, mdl_data=res_mdl, date_list=date_lst, 
                            start='2020-03-12', end='2020-04-05')

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.region_name[region]] = (res_mdl['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
res_mdl['new_Is'].sum()

In [ ]:
ven_proba = pd.read_csv('Data/vent_proba.csv')
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.region_name[region]] = (res_mdl['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

In [ ]:
plt.plot((res_mdl['H'].sum(axis=1))*9136000)

In [ ]:
(res_mdl['H'].sum(axis=1))[56]*9136000

In [ ]:
((res_mdl['Is'][:, mdl.age_dict['70+']].sum(axis=1))[56]+(res_mdl['Is'][:, mdl.age_dict['60-69']].sum(axis=1))[56])/(res_mdl['Is'][:].sum(axis=1))[56]

In [ ]:
(res_mdl['R'].sum(axis=1))[56]*9136000*0.3

### Data with variance

In [ ]:
start_idx = int(np.where(date_lst == '2020-03-12')[0])
end_idx = int(np.where(date_lst == '2020-04-05')[0])

In [ ]:
model_by_age_region = {}
for i, key in enumerate(mdl.region_age_dict.keys()):
    model_by_age_region[key[0] + '_' + key[1] + '_mdl'] = res_mdl['new_Is'][start_idx:end_idx+1, mdl.region_age_dict[key]].sum(axis=1)
model_by_age_region = pd.DataFrame.from_dict(model_by_age_region)

In [ ]:
# Calculating model variance:
std = np.sqrt((np.power(data.values - model_by_age_region.values,2).sum())/(data.values.size - 1))

In [ ]:
std

### Upper bound calibration

In [ ]:
# minimized value: -25.112767900678186
# Fitted parameters:
#  Beta=[0.03030602 0.05262323 0.06149524 0.2050433 ]
#  Theta=2.328913039368483,
#  Beta_behave=0.5297713948054046
# num of sampling the target function: 173

In [ ]:
# Calculating data with adding std
ub_data = data + 2*std

In [ ]:
# fit params
days_in_season_ub=70
alpha_ub=1
date_lst = pd.date_range('2020-02-20','2020-04-13')

In [ ]:
# Model Fitting
p0=(0.06862921,0.07089788,0.08567678,0.3,1.5103006972556037, 0.4293725726208387) # initial guess
bnds = ((0,0.1),(0,0.15),(0,0.15),(0,0.6),(1,3),(0,1))# boundries for variables
# errorfunc(tpl,data,f,season_length, eps, alpha)
res_fit_ub= optimize.minimize(mdl.errorfunc_by_sector_behavior,p0,bounds=bnds,method='TNC',args=(ub_data.values,mdl.f0_full['Scenario2'],
                                                                           days_in_season_ub,mdl.eps_sector['Scenario2'],alpha_ub,date_lst,
                                                                                    '2020-03-12-','2020-04-05')
                           ,options={'maxiter':1000})

fitted_params_ub = res_fit_ub.x

# run the fitted model:
fitted_beta_ub = np.array([fitted_params_ub[0], fitted_params_ub[0], fitted_params_ub[0], fitted_params_ub[1], fitted_params_ub[1],\
                        fitted_params_ub[2], fitted_params_ub[2], fitted_params_ub[3], fitted_params_ub[3]])
theta_ub = fitted_params_ub[4]
beta_behave_ub = fitted_params_ub[5]
                        
# print data stat
mdl.print_stat_fit_behave(res_fit_ub)

In [ ]:
# best fit results:
beta_j_ub=np.array([0.06872524,0.06872524,0.06872524,0.07058089,0.07058089,0.0849962,0.0849962,0.35823708,0.35823708])
theta_ub=1.5147969877267358
beta_behave_ub=0.4195811609779443

In [ ]:
res_mdl_ub = mdl.run_sector_model_behavior(beta_home=0.38/9., theta=theta_ub, beta_j=beta_j_ub, beta_behave=beta_behave_ub,
                                        eps=mdl.eps_sector['Scenario2'], f=mdl.f0_full['Scenario2'], alpha=alpha_ub, 
                                        days_in_season=175,new=1./10.)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=ub_data.values, mdl_data = res_mdl_ub['new_Is'],date_list = date_lst,
                                         start='2020-03-12',end='2020-04-05')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
mdl.plot_calibrated_total_model(data=ub_data, mdl_data=res_mdl_ub, date_list=date_lst, 
                            start='2020-03-12', end='2020-04-05')

In [ ]:
res_mdl_ub['new_Is'].sum()

In [ ]:
plt.plot((res_mdl_ub['H'].sum(axis=1))*9136000)

### Lower bound

In [ ]:
# Calculating data with adding std
lb_data = data*0.5

In [ ]:
# fit params
days_in_season_lb=70
alpha_lb=1
date_lst = pd.date_range('2020-02-20','2020-04-13')

In [ ]:
# Model Fitting
p0=(0.06862921,0.07089788,0.08567678,0.3,1.5103006972556037, 0.4293725726208387) # initial guess
bnds = ((0,0.1),(0,0.15),(0,0.15),(0,0.6),(1,3),(0,1))# boundries for variables
# errorfunc(tpl,data,f,season_length, eps, alpha)
res_fit_lb= optimize.minimize(mdl.errorfunc_by_sector_behavior,p0,bounds=bnds,method='TNC',args=(lb_data.values,mdl.f0_full['Scenario2'],
                                                                           days_in_season_ub,mdl.eps_sector['Scenario2'],alpha_ub,date_lst,
                                                                                    '2020-03-12-','2020-04-05')
                           ,options={'maxiter':1000})

fitted_params_lb = res_fit_lb.x

# run the fitted model:
fitted_beta_lb = np.array([fitted_params_lb[0], fitted_params_lb[0], fitted_params_lb[0], fitted_params_lb[1], fitted_params_lb[1],\
                        fitted_params_lb[2], fitted_params_lb[2], fitted_params_lb[3], fitted_params_lb[3]])
theta_lb = fitted_params_lb[4]
beta_behave_lb = fitted_params_lb[5]
                        
# print data stat
mdl.print_stat_fit_behave(res_fit_lb)

In [ ]:
# best fit results:
beta_j_lb=np.array([0.01684003,0.01684003,0.01684003,0.03004164,0.03004164,0.03680587,0.03680587,0.11561031,0.11561031])
theta_lb=3.0
beta_behave_lb=0.8029761250996457

In [ ]:
res_mdl_lb = mdl.run_sector_model_behavior(beta_home=0.38/9., theta=theta_lb, beta_j=beta_j_lb, beta_behave=beta_behave_lb,
                                        eps=mdl.eps_sector['Scenario2'], f=mdl.f0_full['Scenario2'], alpha=alpha_lb, 
                                        days_in_season=175,new=1./10.)

In [ ]:
# plotting the data and model
fig,ax = mdl.plot_calibrated_model_region(data=lb_data.values, mdl_data = res_mdl_lb['new_Is'],date_list = date_lst,
                                         start='2020-03-12',end='2020-04-05')
fig.set_size_inches((30,30))
plt.tight_layout()
plt.show()

In [ ]:
mdl.plot_calibrated_total_model(data=lb_data, mdl_data=res_mdl_lb, date_list=date_lst, 
                            start='2020-03-12', end='2020-04-05')

In [ ]:
res_mdl_ub['new_Is'].sum()

In [ ]:
plt.plot((res_mdl_lb['H'].sum(axis=1))*9136000)

# 60+ in Quarantine

In [ ]:
inter_age = mdl.get_opposite_dict(mdl.N,list(itertools.product(mdl.M.values(), mdl.A.values())))

In [ ]:
inter_age.keys()

In [ ]:
# age_lst = ['0-4']
age_lst = ['0-4','5-9','10-19','20-29','30-39','40-49','50-59']

In [ ]:
# new S
init_pop = res_mdl_lb['S'][56,:].copy()
for age in mdl.A.values():
#     if age not in ['60-69','70+']:
    if age  in age_lst:
        init_pop[inter_age['Intervention',age]] = init_pop[inter_age['Non-intervention',age]]
        init_pop[inter_age['Non-intervention',age]] = 0

In [ ]:
# new E
init_E = res_mdl_lb['E'][56,:].copy()
for age in mdl.A.values():
#     if age not in ['60-69','70+']:
    if age  in age_lst:
        init_E[inter_age['Intervention',age]] = init_E[inter_age['Non-intervention',age]]
        init_E[inter_age['Non-intervention',age]] = 0

In [ ]:
# new Ie
init_Ie = res_mdl_lb['Ie'][56,:].copy()
for age in mdl.A.values():
#     if age not in ['60-69','70+']:
    if age  in age_lst:
        init_Ie[inter_age['Intervention',age]] = init_Ie[inter_age['Non-intervention',age]]
        init_Ie[inter_age['Non-intervention',age]] = 0

In [ ]:
#new Is
init_Is = res_mdl_lb['Is'][56,:].copy()
for age in mdl.A.values():
#     if age not in ['60-69','70+']:
    if age  in age_lst:
        init_Is[inter_age['Intervention',age]] = init_Is[inter_age['Non-intervention',age]]
        init_Is[inter_age['Non-intervention',age]] = 0

In [ ]:
#new Ia
init_Ia = res_mdl_lb['Ia'][56,:].copy()
for age in mdl.A.values():
#     if age not in ['60-69','70+']:
    if age  in age_lst:
        init_Ia[inter_age['Intervention',age]] = init_Ia[inter_age['Non-intervention',age]]
        init_Ia[inter_age['Non-intervention',age]] = 0

In [ ]:
with open('Data/parameters/C_quarantine_60_70.pickle', 'rb') as pickle_in:
    C = pickle.load(pickle_in)

In [ ]:
res_mdl_60_70 = mdl.continue_run_sector_model_behavior(model_res=res_mdl_lb,t_start=56,intervention=True,C=C,
                                  beta_home=0.38/9., theta=1.0, beta_j=beta_j_lb, beta_behave=beta_behave_lb,
                                        eps=mdl.eps_sector['Scenario2'], f=mdl.f0_full['Scenario2'], alpha=alpha_lb, 
                                        days_in_season=120, new=1/10., new_pop_dist=init_pop, new_E=init_E, new_I_e=init_Ie,
                                                      new_I_s=init_Is, new_I_a=init_Ia)

In [ ]:
res_mdl_60_70['new_Is'].sum()

In [ ]:
plt.plot(res_mdl_60_70['L'][:,mdl.age_ga_dict['50-59']].sum(axis=1))

In [ ]:
res_mdl_60_70['L'][:,mdl.age_ga_dict['70+']].sum(axis=0)

In [ ]:
plt.plot((res_mdl_60_70['H'].sum(axis=1)[:])*9136000)

In [ ]:
# Save
with open('Data/parameters/low_60_70.pickle', 'wb') as handle:
    pickle.dump(res_mdl_60_70, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Creating table 1, by region:
tabel_1_dict = {}
for region in ['11_betshemesh','11','62_arab','51_bb','51','32','24']:
    tabel_1_dict[mdl.region_name[region]] = (res_mdl_60_70['new_Is'][:, mdl.region_dict[region]].sum(axis=1))/mdl.population_size[mdl.region_dict[region]].sum()
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

tabel1_df.iloc[:].plot()

# Data for dashboard

In [ ]:
with open('Data/parameters/low_60_70.pickle', 'rb') as pickle_in:
    res_low = pickle.load(pickle_in)

with open('Data/parameters/high_60_70.pickle', 'rb') as pickle_in:
    res_high = pickle.load(pickle_in)
    
with open('Data/parameters/normal_60_70.pickle', 'rb') as pickle_in:
    res_normal = pickle.load(pickle_in)

In [ ]:
date_rang = pd.date_range('2020-02-20', periods=176, freq='d')

In [ ]:
age_dist_area = pd.read_csv('../Data/demograph/age_dist_area.csv',index_col=0).iloc[0:20][['0-4', '5-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70+']].copy()
age_dist_area = age_dist_area.sum(axis=1)

In [ ]:
tot_I = res_normal['Is'] + res_normal['Ia']
tot_I_ub = res_high['Is'] + res_high['Ia']
tot_I_lb = res_low['Is'] + res_low['Ia']

In [ ]:
# tabel 1, coloring the map
# Creating table 1, by region:
tabel_1_dict = {}
tabel_1_dict['date'] = date_rang
for region in mdl.region_dict.keys():
    tabel_1_dict[mdl.region_name[region]] = (tot_I[:, mdl.region_dict[region]].sum(axis=1))/age_dist_area[region]
tabel1_df = pd.DataFrame.from_dict(tabel_1_dict)

In [ ]:
# 'daysoffset' will container the weekday, as integers
tabel1_df['daysoffset'] = tabel1_df['date'].apply(lambda x: x.weekday())
# We apply, row by row (axis=1) a timedelta operation
tabel1_df['week_start'] = tabel1_df.apply(lambda x: x['date'] - dt.timedelta(days=x['daysoffset']), axis=1)

In [ ]:
tab1_groupped = tabel1_df.groupby('week_start')[list(mdl.region_name.values())]
tab1_groupped = tab1_groupped.apply(lambda x: (x.max() + x.min())/2)

In [ ]:
tab1_groupped[list(mdl.region_name.values())].plot()
plt.hlines(xmin='2020-02-20',xmax='2020-09-30', y=0.0045)
plt.hlines(xmin='2020-02-20',xmax='2020-09-30', y=0.0013)

In [ ]:
tab1_groupped.loc['2020-04-17':'2020-06-17'].stack()

In [ ]:
# Saving table_1:
tab1_groupped.loc['2020-04-17':'2020-06-17'].stack().to_csv('Incidence_per_region_60_70.csv')

In [ ]:
# Age groups

In [ ]:
tabel_2 = pd.DataFrame(index = pd.MultiIndex.from_tuples(itertools.product(date_rang, mdl.A.values()),names=['date','age']), 
                                     columns=['mean','UB','LB'])

In [ ]:
# fill the matrix:
for index in list(tabel_2.index):
#     print(int(np.where(date_rang == index[0])[0]))
    tabel_2.loc[index,'mean'] = tot_I[int(np.where(date_rang == index[0])[0]),mdl.age_dict[index[1]]].sum()
    tabel_2.loc[index,'UB'] = tot_I_ub[int(np.where(date_rang == index[0])[0]),mdl.age_dict[index[1]]].sum()
    tabel_2.loc[index,'LB'] = tot_I_lb[int(np.where(date_rang == index[0])[0]),mdl.age_dict[index[1]]].sum()

In [ ]:
# 'daysoffset' will container the weekday, as integers
tabel_2['daysoffset'] = tabel_2.apply(lambda x: x.name[0].weekday(),axis=1)
# We apply, row by row (axis=1) a timedelta operation
tabel_2['week_start'] = tabel_2.apply(lambda x: x.name[0] - dt.timedelta(days=x['daysoffset']), axis=1)

In [ ]:
tabel_2.reset_index(inplace=True)

In [ ]:
tabel_2_groupped = tabel_2.groupby(['week_start','age'])[['mean','UB','LB']]
tabel_2_groupped = tabel_2_groupped.apply(lambda x: (x.max() + x.min())/2) * 9136000

In [ ]:
tabel_2_groupped['mean'] = tabel_2_groupped['mean'].apply(np.ceil)
tabel_2_groupped['UB'] = tabel_2_groupped['UB'].apply(np.ceil)
tabel_2_groupped['LB'] = tabel_2_groupped['LB'].apply(np.ceil)

In [ ]:
tabel_2_groupped.reset_index(inplace=True)

In [ ]:
tabel_2_groupped[tabel_2_groupped.week_start == '2020-06-15']

In [ ]:
tabel_2_groupped[(tabel_2_groupped.week_start>'2020-04-16')&(tabel_2_groupped.week_start<'2020-06-16')].to_csv('bar_plot_model_60_70.csv')

In [ ]:
# ventilators

In [ ]:
vents_dict['mean'][55]

In [ ]:
vents_dict = {}
vents_dict['date'] = date_rang
vents_dict['mean'] = np.ceil(res_normal['H'].sum(axis=1) * 9136000)+53
vents_dict['UB'] = np.ceil(res_high['H'].sum(axis=1) * 9136000)+53
vents_dict['LB'] = np.ceil(res_low['H'].sum(axis=1) * 9136000)+53
vents_df = pd.DataFrame.from_dict(vents_dict)

In [ ]:
# 'daysoffset' will container the weekday, as integers
vents_df['daysoffset'] = vents_df['date'].apply(lambda x: x.weekday())
# We apply, row by row (axis=1) a timedelta operation
vents_df['week_start'] = vents_df.apply(lambda x: x['date'] - dt.timedelta(days=x['daysoffset']), axis=1)

In [ ]:
vents_df_grop = vents_df.groupby('week_start')[['mean','UB','LB']]
vents_df_grop = vents_df_grop.max()

In [ ]:
vents_df_grop['2020-04-16':'2020-06-16'].to_csv('ventilators_60_70.csv')